<a href="https://colab.research.google.com/github/h-lu/ChatGPT-Next-Web/blob/main/Multi_Agent_with_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q crewai crewai_tools langchain langchain_community langchain-groq

In [ ]:
import os
os.environ["groq_api_key"] = "Your groq api key" # 免费注册获取 https://console.groq.com/keys
os.environ["SERPER_API_KEY"] = "Your Serper api key"#免费注册申请 https://serper.dev/api-key

In [ ]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from crewai_tools import SerperDevTool

## Multi-Agent with CrewAI

作为在 Langchain 的基础之上 的封装，CrewAI 是一个非常热门的 Agent 应用框架，尤其是在 Multi-Agent的场景中，个人感觉比 直接使用 Langchain 模块更清晰也更易用。

Agent 的协同合作提供了很多可能性，CrewAI 就是设计来让 Agent 扮演不同的角色，像一个默契的团队一样，无缝地共同工作，处理复杂任务，拆解子任务，共同完成整体性目标。

- Agent：独立的执行单元，基于特点角色去完成任务，做决策以及和其他 Agent 交互。Agent 可以使用工具来与外部系统交换信息。
- Task：任务就是 Agent 需要完成的任务。这些可能包括任务描述，由哪个 Agent 做这个任务以及借助什么工具，期望什么结果。
- Crew：一个团队就是一群 Agent 各司其职，每个都有特定的角色，合作达成一个共同的目标。组建一个团队的过程包括集合代理，定义任务，建立执行的顺序。

---

本示例使用 CrewAI 来测试 Multi—Agent的协同完成任务的能力：

- 创建一个做计划的 Agent，完成一类计划任务

- 创建一个写作的 Agent，完成一类写作任务

- 创建一个编辑 Agent，完成一类编辑任务

- 创建一个 AI 团队，完成一个综合任务

任务内容：写一篇关于：“angGraph、Autogen和Crewai构建多智能体系统的比较研究” 的文章

---
欢迎关注我的[小红书：AGI悟道](https://www.xiaohongshu.com/user/profile/64cdc8b0000000000b006bff?xhsshare=WeixinSession&appuid=64cdc8b0000000000b006bff&apptime=1715318887&wechatWid=5cd9ba53a281612dd8c33e00d51b9aad&wechatOrigin=menu)，发现更多有趣的AI Demo～


### 模型选择 Groq 提供的 mixtral-8x7b
因为 CrewAI 是基于 Langchain 的 封装，可以直接用 Langchain 的能力

In [ ]:
llm = ChatGroq(temperature=0, groq_api_key=os.environ["groq_api_key"], model_name="mixtral-8x7b-32768")

## Multi-Agent

### 创建一个做计划的 Agent
目标是：Plan engaging and factually accurate content on {topic}

做大纲，标题结构，并搜索内容

In [ ]:
search_tool = SerperDevTool()

planner = Agent(
    # 角色
    role="Content Planner",
    # 目标
    goal="Plan engaging and factually accurate content on {topic}",
    # 故事板
    backstory="""You're working on planning a blog article.
              You collect information that helps the audience learn something and make informed decisions.
              You have to prepare a detailed outline and the relevant topics and sub-topics that has to be a part of the
              blogpost.
              You can search detailed information with tools.
              Your work is the basis for the Content Writer to write an article on this topic.""",
    llm=llm,
    # 是否可以代理到其他 Agent
    allow_delegation=False,
    verbose=True,
    # 工具
    tools=[search_tool],
)


### 创建一个写作的 Agent
目标是：Write insightful and factually accurate opinion piece about the topic: {topic}

按照科技类风格写内容

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",
    backstory="""You're working on a writing a new opinion piece.
              You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.
              You follow the main objectives and direction of the outline, as provide by the Content Planner.
              You also provide objective and impartial insights and back them up with information provide by the Content Planner.
              You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.""",
    llm=llm,
    allow_delegation=False,
    verbose=True,

)

### 创建一个编辑 Agent
目标是：Edit a given blog post to align with the writing style of the organization 'https://medium.com/'

为了发布做校验工作

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writing style of the organization 'https://medium.com/'. ",
    backstory="""You are an editor who receives a blog post from the Content Writer.
              Your goal is to review the blog post to ensure that it follows medium.com best practices,
              provides balanced viewpoints when providing opinions or assertions,
              and also avoids major controversial topics or opinions when possible.""",
    llm=llm,
    allow_delegation=False,
    verbose=True,
)

## 任务

### 创建一个计划任务
任务描述，期望的输出

In [ ]:
plan = Task(
    description="""1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
                   2. Identify the target audience, considering their interests and pain points.
                   3. Develop a detailed content outline including an introduction, key points, and a call to action.
                   4. Search factually accurate detailed information online.
                   5. Include SEO keywords and relevant data or sources.""",
    expected_output="""A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.""",
    agent=planner,
)


### 创建一个写作任务
任务描述，期望的输出

In [ ]:
write = Task(
    description="""1. Use the content plan to craft a compelling blog post on {topic}.
           2. Incorporate SEO keywords naturally.
           3. Sections/Subtitles are properly named in an engaging manner.
           4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
           5. Proofread for grammatical errors and alignment with the brand's voice.""",
    expected_output="""A well-written blog post in markdown format, ready for publication,
                       each section should have 2 or 3 paragraphs.""",
    agent=writer,
)

### 创建一个编辑任务
任务描述，期望的输出

In [ ]:
edit = Task(
    description="""1. Proofread the given blog post for grammatical errors
                   2. Alignment with the brand's voice.
                   3. Align with the writing style of medium.com""",
    expected_output="""A well-written blog post in markdown format, ready for publication,
                       each section should have 2 or 3 paragraphs.""",
    agent=editor
)

## Crew

###创建一个 AI 团队

三个角色，三类任务

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

### 测试

任务主题：Comparative study of LangGraph, Autogen and Crewai for building multi-agent system.


In [ ]:
inputs = {"topic":"Comparative study of LangGraph, Autogen and Crewai for building multi-agent system."}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Comparative study of LangGraph, Autogen and Crewai for building multi-agent system..
                   2. Identify the target audience, considering their interests and pain points.
                   3. Develop a detailed content outline including an introduction, key points, and a call to action.
                   4. Search factually accurate detailed information online.
                   5. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
To complete this task, I will first prioritize the latest trends, key players, and noteworthy news on the comparative study of LangGraph, Autogen, and Crewai for building multi-agent systems. Then, I will identify the target audience and develop a detailed content outline, including an introduction, key points, and a call to action. I will also ensure to include S